In [3]:
import yaml
import os
import pathlib


ROOT = pathlib.Path().resolve().parent

print("Set as base folder: ", ROOT)
os.chdir(ROOT)


Set as base folder:  /Users/andrey/PycharmProjects/DataMiningProjects


In [6]:
ROOT = pathlib.Path().resolve().parent

with open(ROOT / "config.yml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [45]:
#3
# Variables from config
API_KEY = config['api']['key']
API_VERSION = config['api']['api_version']
API_SERVICE_NAME = config['api']['api_service_name']

publishedAfter = config['api']['parameters']['date_after']
publishedBefore = config['api']['parameters']['date_before']



In [46]:
# 4

import googleapiclient.discovery as api

In [47]:
# 5

youtube = api.build(
    API_SERVICE_NAME,
    API_VERSION,
    developerKey = API_KEY
)

In [48]:
# 6
# Одинарный запрос к API
import pandas as pd

# Для поиска
q = "it & (войти | переход | истории)"
search_type = "channel"


request = youtube.search().list(
    part="snippet",
    q=q,
    type=search_type,
    maxResults = 50,
    regionCode="RU"
)

response = request.execute()

response

{'kind': 'youtube#searchListResponse',
 'etag': 'SqGTAJVzph6KR9Q-9kebObqVkXo',
 'nextPageToken': 'CDIQAA',
 'regionCode': 'RU',
 'pageInfo': {'totalResults': 12911, 'resultsPerPage': 50},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'Qj1RsdaSYVkoKjHdgrfksh6O9xU',
   'id': {'kind': 'youtube#channel', 'channelId': 'UCkPnjcP8dPR6wAIzaUdBheQ'},
   'snippet': {'publishedAt': '2019-12-15T22:00:39Z',
    'channelId': 'UCkPnjcP8dPR6wAIzaUdBheQ',
    'title': 'Войти в IT',
    'description': 'Приветствую всех на моем канале! Меня зовут Таня, я сменила несколько профессий в IT (тестировщик, разработчик и ...',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/O8zOJQCrQej6IQtA2lB4So381ZF6l-hal4GazwKrXoxXB0HD0q52qC9ahSl1Aj57aDYLvSl22g=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/O8zOJQCrQej6IQtA2lB4So381ZF6l-hal4GazwKrXoxXB0HD0q52qC9ahSl1Aj57aDYLvSl22g=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/O8zOJQCrQej6IQtA

In [31]:
# 7
df_supplemented = pd.json_normalize(response['items'])

In [32]:
# 8
# Цикл для прохода по всем следующим страницам с выдачей:
i = 0
while 'nextPageToken' in response.keys():
    request = youtube.search().list(
            part ="snippet",
            q = q,
            type = search_type,
            maxResults=50,
            pageToken = response['nextPageToken']
        )
    response = request.execute()

    print(f'Итерация БЕЗ включения аргумента order №{i}')
    i += 1

    df_additional = pd.json_normalize(response['items'])
    df_supplemented = pd.concat([df_supplemented, df_additional])



print(f"Искомых объектов {response['pageInfo']['totalResults']}", \
      f"а найденных БЕЗ включения каких-либо значений аргумента order {len(df_supplemented.drop_duplicates('id.channelId'))}")

Итерация БЕЗ включения аргумента order №0
Итерация БЕЗ включения аргумента order №1
Итерация БЕЗ включения аргумента order №2
Итерация БЕЗ включения аргумента order №3
Итерация БЕЗ включения аргумента order №4
Итерация БЕЗ включения аргумента order №5
Итерация БЕЗ включения аргумента order №6
Итерация БЕЗ включения аргумента order №7
Итерация БЕЗ включения аргумента order №8
Итерация БЕЗ включения аргумента order №9
Искомых объектов 47294 а найденных БЕЗ включения каких-либо значений аргумента order 449


In [33]:
# 9
df_supplemented = df_supplemented.drop_duplicates()

df_supplemented = df_supplemented.reset_index(drop=True)

print('Уникальных объектов', len(df_supplemented))

Уникальных объектов 456


In [36]:
# 9
df_supplemented.to_excel('data/news_search_result_inorder.xlsx')